In [1]:
from pathlib import Path

import numpy as np
import pandas as pd


pd.set_option("display.max_columns", 100)

from google.colab import drive
drive.mount('/content/gdrive')
DATA_PATH = "/content/gdrive/My Drive/MLdata/"

train_df = pd.read_csv(
    DATA_PATH + "train.csv", 
    index_col="tripid"
)
test_df = pd.read_csv(
    DATA_PATH + "test.csv", 
    index_col="tripid"
)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
train_df["label"] = train_df["label"].map({"incorrect": 0, "correct":1})

In [3]:
train_df["pickup"] = pd.to_datetime(train_df["pickup_time"])
train_df["drop"] = pd.to_datetime(train_df["drop_time"])
train_df ['difference'] = train_df["drop"] - train_df["pickup"]
train_df ['difference'] = train_df ['difference']/np.timedelta64(1,'s')
train_df['duration'] = train_df['duration'].fillna(train_df ['difference'])
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   additional_fare            16974 non-null  float64       
 1   duration                   17176 non-null  float64       
 2   meter_waiting              16974 non-null  float64       
 3   meter_waiting_fare         16974 non-null  float64       
 4   meter_waiting_till_pickup  16974 non-null  float64       
 5   pickup_time                17176 non-null  object        
 6   drop_time                  17176 non-null  object        
 7   pick_lat                   17176 non-null  float64       
 8   pick_lon                   17176 non-null  float64       
 9   drop_lat                   17176 non-null  float64       
 10  drop_lon                   17176 non-null  float64       
 11  fare                       17039 non-null  float64     

In [4]:
train_df.head()

# new_df = train_df["pickup_time"]-train_df["drop_time"]
# train_df.info()
# train_df.head()
train_df.isnull().sum()
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,pickup,drop,difference
tripid,,,,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,1,2019-11-01 00:20:00,2019-11-01 00:34:00,840.0
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,1,2019-11-01 00:56:00,2019-11-01 01:09:00,780.0
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,1,2019-11-01 01:08:00,2019-11-01 01:26:00,1080.0
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,1,2019-11-01 02:27:00,2019-11-01 02:37:00,600.0
189128020,NaN,1020.0,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,1,2019-11-01 03:34:00,2019-11-01 03:51:00,1020.0


In [5]:
train_df2 = train_df.drop(['pickup_time','drop_time','pickup','drop','difference'],axis='columns')
train_df2.isnull().sum()

additional_fare              202
duration                       0
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64

In [ ]:
train_df2= train_df2.dropna()

In [6]:
from math import radians, cos, sin, asin, sqrt
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # 6371 is Radius of earth in kilometers. Use 3956 for miles
    return km

train_df2['distance'] = haversine_np(train_df2['pick_lat'], train_df2['pick_lon'], train_df2['drop_lat'] , train_df2['drop_lon'])

In [8]:
train_df2['distance_per_fare'] = train_df2['distance']/train_df2['fare']
train_df2.loc[~np.isfinite(train_df2['distance_per_fare'] ), 'distance_per_fare'] = 0

In [9]:
train_df2['absoluteduration_per_fare'] = (train_df2['duration']- train_df2['meter_waiting'])/train_df2['fare']
train_df2.loc[~np.isfinite(train_df2['absoluteduration_per_fare'] ), 'absoluteduration_per_fare'] = 0

In [10]:
clean_df = train_df2.drop(['pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')

In [11]:

clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            16974 non-null  float64
 1   duration                   17176 non-null  float64
 2   meter_waiting              16974 non-null  float64
 3   meter_waiting_fare         16974 non-null  float64
 4   meter_waiting_till_pickup  16974 non-null  float64
 5   fare                       17039 non-null  float64
 6   label                      17176 non-null  int64  
 7   distance                   17176 non-null  float64
 8   feature1                   17176 non-null  float64
 9   distance_per_fare          17176 non-null  float64
 10  absoluteduration_per_fare  17176 non-null  float64
dtypes: float64(10), int64(1)
memory usage: 1.6 MB


In [12]:
clean_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,distance,feature1,distance_per_fare,absoluteduration_per_fare
tripid,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,270.32,1,2.467063,0.009126,0.009126,2.878070
189125358,10.5,791.0,47.0,0.0000,134.0,197.85,1,0.868942,0.004392,0.004392,3.760425
189125719,10.5,1087.0,80.0,0.0000,61.0,301.64,1,5.531798,0.018339,0.018339,3.338417
189127273,10.5,598.0,271.0,15.6638,68.0,82.30,1,0.845795,0.010277,0.010277,3.973269
189128020,NaN,1020.0,NaN,NaN,NaN,358.39,1,7.527884,0.021005,0.021005,0.000000


In [13]:
clean_df = clean_df.drop(['meter_waiting_till_pickup','meter_waiting'],axis='columns')

In [14]:
clean_df.head(20)

,additional_fare,duration,meter_waiting_fare,fare,label,distance,feature1,distance_per_fare,absoluteduration_per_fare
tripid,,,,,,,,,
189123628,10.5,834.0,0.000000,270.32,1,2.467063,0.009126,0.009126,2.878070
189125358,10.5,791.0,0.000000,197.85,1,0.868942,0.004392,0.004392,3.760425
189125719,10.5,1087.0,0.000000,301.64,1,5.531798,0.018339,0.018339,3.338417
189127273,10.5,598.0,15.663800,82.30,1,0.845795,0.010277,0.010277,3.973269
189128020,NaN,1020.0,NaN,358.39,1,7.527884,0.021005,0.021005,0.000000
189129552,10.5,3407.0,0.000000,1065.02,1,5.510397,0.005174,0.005174,3.028112
189132829,10.5,1246.0,0.000000,266.62,1,4.749980,0.017816,0.017816,2.846748
189135103,10.5,1333.0,17.198500,318.05,1,5.359651,0.016852,0.016852,3.263638
189139296,10.5,360.0,4.664000,100.32,1,0.679699,0.006775,0.006775,2.791069


**Training**

In [15]:

X_train = clean_df.drop(['label'],axis='columns')
Y_train = clean_df['label']
Y_train.head()

tripid
189123628    1
189125358    1
189125719    1
189127273    1
189128020    1
Name: label, dtype: int64

In [16]:
from pathlib import Path
from math import radians, cos, sin, asin, sqrt
import datetime as dt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier


In [17]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numeric_features = clean_df.select_dtypes(include=['int64', 'float64']).drop(['label'], axis=1).columns

categorical_features = clean_df.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# get the models to evaluate
level0 = list()
level0.append(('xg',XGBClassifier(random_state=2, max_depth=4,n_estimators=300,gamma=0.3,learning_rate=0.2,max_delta_step=1))) 
level0.append(('mlp',MLPClassifier(hidden_layer_sizes=(50,100,50), max_iter=1000)))
level0.append(('dt', RandomForestClassifier(n_estimators = 100,max_features = 'log2')))

# define meta learner model
level1 = LogisticRegression(penalty="l2", C=3)
 
estimator = StackingClassifier(estimators=level0, final_estimator=level1, cv=10)

fullPipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', estimator)])

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_eval, y_train, y_eval = train_test_split(X_train, Y_train, test_size=0.33, shuffle=True, stratify=Y_train, random_state=6)


In [20]:

# Train model
fullPipe.fit(x_train, np.ravel(y_train))

preds = fullPipe.predict(x_eval)
y_preds = pd.DataFrame({"label": preds},index = y_eval.index)

from sklearn.metrics import f1_score
print(f1_score(y_eval, y_preds))

0.9749228395061729


In [21]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_eval, y_preds)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.813953570308674

In [ ]:
fullPipe.fit(X_train, np.ravel(Y_train))

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='mean',
                                                               

**Predicting**

In [ ]:
test_df.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
dtype: int64

In [ ]:

test_df.info()
test_df.head()
test_df.isnull().sum()
test_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8576 entries, 213284604 to 222860703
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            8576 non-null   float64
 1   duration                   8576 non-null   int64  
 2   meter_waiting              8576 non-null   int64  
 3   meter_waiting_fare         8576 non-null   float64
 4   meter_waiting_till_pickup  8576 non-null   int64  
 5   pickup_time                8576 non-null   object 
 6   drop_time                  8576 non-null   object 
 7   pick_lat                   8576 non-null   float64
 8   pick_lon                   8576 non-null   float64
 9   drop_lat                   8576 non-null   float64
 10  drop_lon                   8576 non-null   float64
 11  fare                       8576 non-null   float64
dtypes: float64(7), int64(3), object(2)
memory usage: 871.0+ KB


,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [ ]:
test_df['distance'] = haversine_np(test_df['pick_lat'], test_df['pick_lon'], test_df['drop_lat'] , test_df['drop_lon'])
test_df2 = test_df.drop(['pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon'],axis='columns')

test_df2.isnull().sum()

additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
fare                         0
distance                     0
dtype: int64

In [ ]:
test_df2['distance_per_fare'] = test_df2['distance']/test_df2['fare']
test_df2.loc[~np.isfinite(test_df2['distance_per_fare'] ), 'distance_per_fare'] = 0

test_df2['absoluteduration_per_fare'] = (test_df2['duration']- test_df2['meter_waiting'])/test_df2['fare']
test_df2.loc[~np.isfinite(test_df2['absoluteduration_per_fare'] ), 'absoluteduration_per_fare'] = 0

test_df2 = test_df2.drop(['meter_waiting_till_pickup','meter_waiting'],axis='columns')

In [ ]:
result = fullPipe.predict(test_df2)
test_df2['prediction'] = result

In [ ]:
result

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
final_df = test_df2.drop(['additional_fare','distance','duration','fare','feature2','feature1','meter_waiting_fare'],axis='columns')

final_df.head()
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8576 entries, 213284604 to 222860703
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   prediction  8576 non-null   int64
dtypes: int64(1)
memory usage: 134.0 KB


In [ ]:
final_df

,prediction
tripid,
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
...,...
222856243,1
222857785,1
222858416,1


In [ ]:
final_df.to_csv(DATA_PATH+'submission_emsemble.csv', index=True)